We are looking for card ids:
    
    1. present in historical transaction
    
    2. not present in new transaction
    
    3. also has corresponding merchant information



In [2]:
import pandas as panda
import numpy as np
from collections import Counter

In [21]:
train_data = panda.read_csv('train_2.csv')
historical_data_with_merchant = panda.read_csv('hist_joined_with_merchants_data.csv')
new_data_with_merchant = panda.read_csv('new_with_merchant_aggregated.csv')
hist_data_agg = panda.read_csv('hist_with_merchant_aggregated.csv')
merchant_data = panda.read_csv('merchants_2.csv')

In [4]:
hist_data_agg.shape, hist_data_agg.card_id.unique().shape,merchant_data.shape, merchant_data.merchant_id.unique().shape,

((325540, 32), (325540,), (334696, 34), (334633,))

In [5]:
# result = panda.merge(hist_data_agg, merchant_data, on='merchant_id', how='inner')
# result = hist_data_agg.join(merchant_data, how='left', on='merchant_id')
result = panda.concat([hist_data_agg, merchant_data], axis=1, join ='inner')
print('before drop:', result.shape)
result.dropna(inplace=True)
print('after drop:', result.shape, result.card_id.unique().shape)


before drop: (325540, 66)
after drop: (325540, 66) (325540,)


In [6]:
historical_data_with_merchant.shape,historical_data_with_merchant.card_id.unique().shape, result.shape, result.card_id.unique().shape


((347585, 65), (325540,), (325540, 66), (325540,))

In [8]:

train_data_card_ids = set(train_data.card_id.values.tolist())
new_trn_data_card_ids = set(new_data_with_merchant.card_id.values.tolist())

card_ids_with_no_new_transaction = [i for i in train_data_card_ids if i not in new_trn_data_card_ids]
len(card_ids_with_no_new_transaction)

22517

In [18]:
result[result.card_id.isin(card_ids_with_no_new_transaction)].shape, result.isnull().any().sum()

((22517, 66), 0)

In [22]:
train_data_1 = train_data[train_data.card_id.isin(card_ids_with_no_new_transaction)]

In [23]:
train_data_1.isnull().sum()

card_id             0
target              0
enc_feature_1_1     0
enc_feature_1_2     0
enc_feature_1_3     0
enc_feature_1_4     0
enc_feature_1_5     0
months_passed       0
first_active_day    0
dtype: int64

In [24]:
train_data_1.shape, train_data_1.card_id.unique().shape, result.shape, result.card_id.unique().shape

((22517, 9), (22517,), (325540, 66), (325540,))

In [25]:
temp = panda.merge(train_data_1, result, on='card_id', how='inner')
print('before dropping:', temp.shape)
temp.dropna(inplace=True)
print('after dropping:', temp.shape)

before dropping: (22517, 74)
after dropping: (22517, 74)


In [47]:
temp.to_csv('train_with_only_hist_with_merchant.csv',index=False)

In [27]:
test_data = panda.read_csv('test_2.csv')
test_data.shape

(123623, 8)

In [28]:

test_data_card_ids = set(test_data.card_id.values.tolist())
new_trn_data_card_ids = set(new_data_with_merchant.card_id.values.tolist())

test_card_ids_with_no_new_transaction = [i for i in test_data_card_ids if i not in new_trn_data_card_ids]
len(test_card_ids_with_no_new_transaction)

13966

In [29]:
result[result.card_id.isin(test_card_ids_with_no_new_transaction)].shape, result.isnull().any().sum()

((13966, 66), 0)

In [30]:
test_data_1 = test_data[test_data.card_id.isin(test_card_ids_with_no_new_transaction)]
test_data_1.shape, test_data_1.card_id.unique().shape

((13966, 8), (13966,))

In [31]:
temp_1 = panda.merge(test_data_1, result, on='card_id', how='inner')
print('before dropping:', temp_1.shape)
temp_1.dropna(inplace=True)
print('after dropping:', temp_1.shape)

before dropping: (13966, 73)
after dropping: (13966, 73)


In [48]:
temp_1.to_csv('test_with_only_hist_with_merchant.csv',index=False)

In [34]:
historical_data_no_merchant = panda.read_csv('hist_without_merchant_aggregated.csv')
historical_data_no_merchant.shape,historical_data_no_merchant.card_id.unique().shape,

((69254, 23), (69254,))

In [38]:
train_card_id_in_history_without_merchant =set(train_data.card_id.values.tolist()).intersection(set(historical_data_no_merchant.card_id.values.tolist()))

In [39]:
len(train_card_id_in_history_without_merchant)

42885

In [42]:
test_card_id_in_history_without_merchant =set(test_data.card_id.values.tolist()).intersection(set(historical_data_no_merchant.card_id.values.tolist()))

In [43]:
len(test_card_id_in_history_without_merchant)

26369

In [44]:
test_data_2 = test_data[test_data.card_id.isin(test_card_id_in_history_without_merchant)]
test_data_2.shape, test_data_2.card_id.unique().shape

((26369, 8), (26369,))

In [45]:
train_data_2 = test_data[train_data.card_id.isin(train_card_id_in_history_without_merchant)]
train_data_2.shape, train_data_2.card_id.unique().shape

C:\Users\somak\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


((26257, 8), (26257,))

In [46]:
temp_2 = panda.merge(test_data_2, result, on='card_id', how='inner')
print('before dropping:', temp_2.shape)
temp_2.dropna(inplace=True)
print('after dropping:', temp_2.shape)

before dropping: (26369, 73)
after dropping: (26369, 73)


In [49]:
temp_2.to_csv('test_with_only_hist_with_no_merchant.csv',index=False)

In [50]:
temp_3 = panda.merge(train_data_2, result, on='card_id', how='inner')
print('before dropping:', temp_3.shape)
temp_3.dropna(inplace=True)
print('after dropping:', temp_3.shape)

before dropping: (26257, 73)
after dropping: (26257, 73)


In [51]:
temp_3.to_csv('train_with_only_hist_with_no_merchant.csv',index=False)